In [1]:
import coremltools
from coremltools.proto import FeatureTypes_pb2
from src.datasets import camvid
from src.tiramisu import tiramisu

# Model

In [2]:
# load the dataset to get the output shape information
camvid11 = camvid.CamVid(mapping=camvid.CamVid.load_mapping(), ignored_labels=['Void'])
# build the model for the image shape and number of labels
model = tiramisu.tiramisu((352, 480, 3), camvid11.n,
    class_weights=camvid11.class_mask,
    growth_rate=12,
    layer_sizes=[3, 3, 3, 3, 3],
    bottleneck_size=3,
)
model.load_weights('models/Tiramisu45-CityScapes.h5')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Tiramisu_input (InputLayer)     (None, 352, 480, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 352, 480, 48) 1344        Tiramisu_input[0][0]             
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 352, 480, 48) 192         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 352, 480, 48) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

# CoreML Tools

In [3]:
coreml_model = coremltools.converters.keras.convert(model,
	input_names='image',
	image_input_names='image',
    output_names='segmentation',
	image_scale=1/255.0,
)

0 : Tiramisu_input, <keras.engine.topology.InputLayer object at 0x123ee15f8>
1 : conv2d_1, <keras.layers.convolutional.Conv2D object at 0x123f10438>
2 : batch_normalization_1, <keras.layers.normalization.BatchNormalization object at 0x123f10550>
3 : activation_1, <keras.layers.core.Activation object at 0x123e3c710>
4 : conv2d_2, <keras.layers.convolutional.Conv2D object at 0x1149a7550>
5 : concatenate_1, <keras.layers.merge.Concatenate object at 0x123f4c588>
6 : batch_normalization_2, <keras.layers.normalization.BatchNormalization object at 0x123fb3a58>
7 : activation_2, <keras.layers.core.Activation object at 0x123fb3e48>
8 : conv2d_3, <keras.layers.convolutional.Conv2D object at 0x124039ba8>
9 : concatenate_2, <keras.layers.merge.Concatenate object at 0x1240b4400>
10 : batch_normalization_3, <keras.layers.normalization.BatchNormalization object at 0x1240b48d0>
11 : activation_3, <keras.layers.core.Activation object at 0x1240b47b8>
12 : conv2d_4, <keras.layers.convolutional.Conv2D obj

110 : activation_26, <keras.layers.core.Activation object at 0x125806a20>
111 : conv2d_27, <keras.layers.convolutional.Conv2D object at 0x125892f98>
112 : concatenate_27, <keras.layers.merge.Concatenate object at 0x1258f5b00>
113 : conv2d_transpose_2, <keras.layers.convolutional.Conv2DTranspose object at 0x1258f5fd0>
114 : concatenate_28, <keras.layers.merge.Concatenate object at 0x1259a6e80>
115 : batch_normalization_27, <keras.layers.normalization.BatchNormalization object at 0x1259a6e10>
116 : activation_27, <keras.layers.core.Activation object at 0x1259a6630>
117 : conv2d_28, <keras.layers.convolutional.Conv2D object at 0x125a0bc88>
118 : concatenate_29, <keras.layers.merge.Concatenate object at 0x125a66208>
119 : batch_normalization_28, <keras.layers.normalization.BatchNormalization object at 0x125a42e48>
120 : activation_28, <keras.layers.core.Activation object at 0x125a42e80>
121 : conv2d_29, <keras.layers.convolutional.Conv2D object at 0x125b0fc88>
122 : concatenate_30, <keras.

In [4]:
coreml_model.author = 'Kautenja'
coreml_model.license = 'MIT'
coreml_model.short_description = '45 Layers Tiramisu Semantic Segmentation Model trained on CamVid & CityScapes.'
coreml_model.input_description['image'] = 'An input image in RGB order'
coreml_model.output_description['segmentation'] = 'The segmentation map as the Softmax output'

In [5]:
coreml_model.save('models/Tiramisu45.mlmodel')

## Convert Outputs to Float32

In [6]:
# get the spec from the model
spec = coreml_model.get_spec()
# create a local reference to the Float32 type
Float32 = FeatureTypes_pb2.ArrayFeatureType.FLOAT32
# set the output shape for the segmentation to Float32
spec.description.output[0].type.multiArrayType.dataType = Float32
# visualize the spec to validate changes
spec.description

input {
  name: "image"
  shortDescription: "An input image in RGB order"
  type {
    imageType {
      width: 480
      height: 352
      colorSpace: RGB
    }
  }
}
output {
  name: "segmentation"
  shortDescription: "The segmentation map as the Softmax output"
  type {
    multiArrayType {
      shape: 12
      shape: 352
      shape: 480
      dataType: FLOAT32
    }
  }
}
metadata {
  shortDescription: "45 Layers Tiramisu Semantic Segmentation Model trained on CamVid & CityScapes."
  author: "Kautenja"
  license: "MIT"
}

In [7]:
coremltools.utils.save_spec(spec, 'models/Tiramisu45.mlmodel')